In [125]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from itertools import product

In [126]:
DOMAIN = "https://ibjjf.com"
URL = f"{DOMAIN}/2024-athletes-ranking"
print(URL)

https://ibjjf.com/2024-athletes-ranking


In [127]:
PARAMETERS = {
  'utf-8':'✓',
  'filters[s]': 'ranking-geral-gi',
  'filters[ranking_category]': 'master',
  'filters[gender]': 'male',
  'filters[age_division]': 'master1',
  'filters[belt]': 'black',
  'filters[weight]': None,
  'page': 1
}
HEADERS = {'User-Agent': 'Mozilla/5.0'}


In [128]:
response = requests.get(url = URL,params = PARAMETERS, headers= HEADERS)
if response.status_code == 200:
  print("Requisicao feita com sucesso")
else:
  print("Requisicao Incompleta. Erro")

Requisicao feita com sucesso


In [129]:
response = response.text

### Aqui entra a Beautiful Soup, ela vai nos ajudar na organização da página puxada

In [130]:
soup = BeautifulSoup(response, 'html.parser')

In [131]:
def get_page_content(url,parameters,headers):
  response = requests.get(url = url ,params= parameters, headers= headers).text
  soup = BeautifulSoup(response, 'html.parser')
  return soup

In [132]:
athletes = []

In [133]:
def parse_athletes (soup, kimono, category, belt, divison, gender):
  try:
    table = soup.find('table')
    rows = table.find_all('tr')
    
    for row in rows:
      if row != None:
        # Bloco de Celulas
        photo_cell = row.find('td', class_ = "photo reduced" )
        if photo_cell != None:
          # photo_tag = row.find('td', class_ = "photo reduced" ).find('img')['src']
          pontuations = row.find('td', class_ = "pontuation" )
          name_tag = row.find('div', class_ = 'name').find('a')
          rank_cell = row.find('td', class_ = 'position')
          
          # Bloco que trabalha com os valores das células
          name = name_tag.get_text(strip = True)
          points = pontuations.get_text(strip = True)
          details = DOMAIN + name_tag['href']
          ranking = rank_cell.get_text(strip = True)
          
          # Ideia, acrescentar categoria, divisao, faixa, gênero etc
          
          
          athlete = {
            'nome':name, 'points':points, 'ranking': ranking, 'details':details, 'kimono':kimono, 'category': category, 'belt': belt, 'division': divison, 'gender': gender
            }
          
          if athlete is None:
            break
          else:
            athletes.append(athlete)
  except:
    print('Linhas ou Tabelas não encontradas')


In [134]:
# imprimindo valores com map
def printValores(objeto):
  print(f"{objeto['nome']}, Pontos: {objeto['points']}, Ranking: {objeto['ranking']}, Detalhes:{objeto['details']}")
x = map(printValores, athletes)

In [135]:
def list_filters(soup, filter_id):
  filter = soup.find(id = filter_id).findAll('option')
  return [item['value'] for item in filter[1:]]

In [136]:
soup_filters = get_page_content(URL,PARAMETERS,HEADERS)

In [137]:
# Coletando filtros
division_options = list_filters(soup_filters, 'weight_filter')
kimono_options = list_filters(soup_filters, 'filters_s')
gender_options = list_filters(soup_filters, 'filters_gender')
belt_options = list_filters(soup_filters,'filters_belt')
category_options = list_filters(soup_filters, 'filters_ranking_category')

In [138]:
for a,b,c,d,e in product(kimono_options,division_options,gender_options, belt_options, category_options):
  print(f'Scrapping: {a} , {b} , {c}, {d} , {e}, page: {i}')
# Coletando de Página em  Página
  for i in range(1,2):
    PARAMETERS['page'] = i
    PARAMETERS['filters[s]'] = a
    PARAMETERS['filters[ranking_category]'] = e
    PARAMETERS['filters[gender]']= c
    PARAMETERS['filters[belt]']= d
    PARAMETERS['filters[weight]']= b
    soup_athletes = get_page_content(URL,PARAMETERS,HEADERS)
    parse_athletes(soup_athletes,a,e,d,b,c)

Scrapping: ranking-geral-gi , rooster , male, black , adult, page: 1
Scrapping: ranking-geral-gi , rooster , male, black , master, page: 1
Scrapping: ranking-geral-gi , rooster , male, black , juvenile, page: 1
Linhas ou Tabelas não encontradas
Scrapping: ranking-geral-gi , rooster , male, black , kids, page: 1
Linhas ou Tabelas não encontradas
Scrapping: ranking-geral-gi , rooster , male, brown , adult, page: 1
Scrapping: ranking-geral-gi , rooster , male, brown , master, page: 1
Scrapping: ranking-geral-gi , rooster , male, brown , juvenile, page: 1
Linhas ou Tabelas não encontradas
Scrapping: ranking-geral-gi , rooster , male, brown , kids, page: 1
Linhas ou Tabelas não encontradas
Scrapping: ranking-geral-gi , rooster , male, purple , adult, page: 1
Scrapping: ranking-geral-gi , rooster , male, purple , master, page: 1
Scrapping: ranking-geral-gi , rooster , male, purple , juvenile, page: 1
Linhas ou Tabelas não encontradas
Scrapping: ranking-geral-gi , rooster , male, purple , kid

In [139]:
df_athletes = pd.json_normalize(athletes)
df_athletes

,nome,points,ranking,details,kimono,category,belt,division,gender
0,Thalison Vitorino Soares,534,1,https://ibjjf.com/athletes/thalison-soares-4b8...,ranking-geral-gi,adult,black,rooster,male
1,Lucas dos Santos Pinheiro,261,2,https://ibjjf.com/athletes/lucas-pinheiro,ranking-geral-gi,adult,black,rooster,male
2,Samuel David Marquez,165,3,https://ibjjf.com/athletes/samuel-marquez,ranking-geral-gi,adult,black,rooster,male
3,Everton Celio de Souza Teixeira,162,4,https://ibjjf.com/athletes/everton-teixeira,ranking-geral-gi,adult,black,rooster,male
4,Welerson Gonçalves da Silva,147,5,https://ibjjf.com/athletes/welerson-silva,ranking-geral-gi,adult,black,rooster,male
...,...,...,...,...,...,...,...,...,...
4670,Amber Lee Richards,85.5,6,https://ibjjf.com/athletes/amber-richards,ranking-geral-no-gi,kids,blue,openclass,female
4671,Adriene Félix Franco,76.5,7,https://ibjjf.com/athletes/adriene-franco,ranking-geral-no-gi,kids,blue,openclass,female
4672,Candyce Denise Raiford,67.5,8,https://ibjjf.com/athletes/candyce-raiford,ranking-geral-no-gi,kids,blue,openclass,female
4673,Charmie Dawn Smith,54.0,9,https://ibjjf.com/athletes/charmie-smith,ranking-geral-no-gi,kids,blue,openclass,female


In [140]:
# Gravar Arquivo em .CSV
path = 'Atletas.csv'
df_athletes.to_csv(path, index = False)